# Pauli Objects

## Introduction and Arithmetic

Pauli operators acting on the $N$-qubit Hilbert space are ubiquitous in quantum circuit computations. In domain applications like QAOA and VQE, these often appear with a sparse structure corresponding to the physics of the problem at hand. To efficiently manipulate such operators and compute expectation values over the relevant Pauli strings, `quasar` features a `Pauli` class (and associated utility classes) that provide for sparse representation and intuitive manipulation of Pauli-type objects. 

To get started, let's grab some utility objects that generate the Pauli operators on individual qubit indices:

In [1]:
import quasar
I, X, Y, Z = quasar.Pauli.IXYZ()

Now, we can use these to start building up Pauli operators:

In [2]:
pauli = I[-1] + X[0] - X[0] * Y[1] + 0.5 * X[0] * Y[1] * Z[2] - X[0] * Y[1] * Z[2] / 4.0
pauli += 1.0
pauli += 2.0 * (X[0] + Z[0]) * (X[0] + Y[0])
pauli += pauli
pauli /= 2.0
print(pauli)

+4.0*I
+(1-2j)*X0
+2j*Y0
+2j*Z0
-1.0*X0*Y1
+0.25*X0*Y1*Z2


In [10]:
print(X[0] + X[1] + Z[0] + Z[1])

+1.0*X0
+1.0*Z0
+1.0*X1
+1.0*Z1


How did that work? Well, first off `I` is just a trivial `Pauli` object corresponding to the identity operator acting on all qubits:

In [3]:
print(I)
print(type(I))

<class 'quasar.pauli.PauliStarter'>


Next, `X/Y/Z` are "`PauliStarter`" objects that generate `Pauli` objects when indexed by the `[]` operator:

In [4]:
print(type(X))
print(X[3])
print(type(X[3]))

<class 'quasar.pauli.PauliStarter'>
+1.0*X3
<class 'quasar.pauli.Pauli'>


Next, we have overloaded all relevant arithmetic operators within the `Pauli` class to allow arithmetic composition of linear combinations of products of single-qubit Pauli operators. These work with either `float` or `complex` coefficients:

In [5]:
print(-1.j * X[0])

-1j*X0


Note that products of Pauli operators on the same qubit index are automatically reduced to linear or constant complexity by the Pauli multiplication rules:

In [6]:
print(X[0]*X[0])
print(X[0]*Y[0])
print(X[0]*Z[0])
print(Y[0]*X[0])
print(Y[0]*Y[0])
print(Y[0]*Z[0])
print(Z[0]*X[0])
print(Z[0]*Y[0])
print(Z[0]*Z[0])
print(X[0]*Y[0]*Z[0])
print(Y[0]*X[0]*Z[0])

+1.0*I
+1j*Z0
-1j*Y0
-1j*Z0
+1.0*I
+1j*X0
+1j*Y0
-1j*X0
+1.0*I
+1j*I
-1j*I


## Sympy and Pauli

`Pauli` is designed to play nicely with `sympy`-type coefficients:

In [7]:
import sympy
a, b, c = sympy.symbols('a b c')
print(a*I[-1] + b*X[0] + c**2*Y[1] + a + 1.0*Y[3])

+2.0*a*I
+1.0*b*X0
+1.0*c**2*Y1
+1.0*Y3


This is primarily useful for performing derivations. 

## Pauli Summary Attributes

The number of qubits attribute `N` is simply one larger than the maximum qubit index currently present in the `Pauli` object:

In [8]:
print(pauli.N)

AttributeError: 'Pauli' object has no attribute 'N'

The maximum order attribute is the maximum number of one-qubit Pauli operators currently present in the `Pauli` object:

In [ ]:
print(pauli.max_order)

The number of terms attribute is the number of Pauli strings currently present in the `Pauli` object:

In [ ]:
print(pauli.nterm)

These are easily printable in a handy summary string:

In [ ]:
print(pauli.summary_str)

The qubit indices and characters of each of the Pauli strings can be obtained from the `qubits` and `chars` attributes. These can also be found from the `PauliString` objects in the `Pauli` object keys, as shown below.

In [ ]:
print(pauli.qubits)
print(pauli.chars)

The unique Pauli characters encountered in all strings can be obtained from the `unique_chars` attribute:

In [ ]:
print(pauli.unique_chars)

## Dictionary Structure

The `Pauli` class actually inherits from the `collections.OrderedDict` class, and internally is represented as an `OrderedDict` of `PauliString : float/complex` pairs.

In [ ]:
for string, value in pauli.items():
    print('%-8s : %6s' % (string, value))

We can use `dict` access methods to access and modify the coefficients of existing Pauli strings:

In [ ]:
pauli[quasar.PauliString.from_string('X0*Y1')] += 1.0
print(pauli)

and even add new Pauli strings:

In [ ]:
pauli[quasar.PauliString.from_string('X0*Y4')] = 1.0
print(pauli)

All `dict` access methods have been overloaded to also accept `str` arguments, which will be converted to `PauliString` under the hood:

In [ ]:
pauli['X0'] += 1.0
print(pauli['X0'])
print(pauli['X0*Y1'])

## PauliString and PauliOperator Utility Classes

Utility class `PauliString` represents a string of one or more `PauliOperator` objects:

In [ ]:
string = quasar.PauliString.from_string('X0*Y1*Z2')
print(string)
print(string.order)
print(string.qubits)
print(string.chars)

From a data structure perspective, `PauliString` inherits from `tuple` and contains a `tuple` of `PauliOperator` objects:

In [ ]:
print(string[0])
print(type(string[0]))

Utility class `PauliOperator` represents a one-qubit Pauli operator with a given character and qubit index:

In [ ]:
operator = quasar.PauliOperator.from_string('Y10')
print(operator)
print(operator.qubit)
print(operator.char)

## Symmetry Considerations

In practical problems involving two-qubit and $k$-qubit interactions, one usually encounters one of two possible conventions in the literature/code structure: the upper triangular representation or the symmetric representation. To permit both possibilities, `quasar` considers, e.g., `Z0*Z1` and `Z1*Z0` to be inequivalent `PauliString` objects:

In [ ]:
print(Z[0]*Z[1] + Z[1]*Z[0])

In [ ]:
print(Z[0]*Z[1] + Z[0]*Z[1])

## Additional Math Functions

A few additional math functions are provided in `Pauli` for convenience/advanced users. The `conj` property returns a new version of the `Pauli` object with the values conjugated:

In [ ]:
print(pauli.conj)

The `dot` method takes the dot product of coincident Pauli strings in two `Pauli` objects:

In [ ]:
pauli = 3.0 * I + 4.0 * Z[0] * Z[1]
print(pauli.dot(pauli))

The `norm2` property returns the square root of the dot product of `self` with `self`:

In [ ]:
print(pauli.norm2)

The `norminf` property returns the magnitude of the largest coefficient of `self`:

In [ ]:
print(pauli.norminf)

The `zero` static method returns a new `Pauli` object initialized with no strings:

In [ ]:
print(quasar.Pauli.zero())

The `zeros_like` static method returns a new `Pauli` object initialized with the strings of another `Pauli` object, but with coefficient values initialized to zero:

In [ ]:
print(quasar.Pauli.zeros_like(pauli))

Many times, arithmetic manipulations yield Pauli strings with small or even zero coefficient values. A new `Pauli` object with such strings removed can be obtained by calling the `sieved` method, which takes an optional `cutoff` argument:

In [ ]:
pauli -= 3.0 * I
print(pauli)

In [ ]:
print(pauli.sieved())

In [ ]:
print(pauli.sieved(cutoff=1.0E-14))

## Technical Note

Note that `quasar` provides efficient incremental add/subtract functions for `Pauli` that do not copy the whole `Pauli` object on the left-hand-side of the expression, but care must be taken to ensure that the right-hand-sides are not modified when using this.  E.g., some care is needed when initializing a new `Pauli` object with an `I`-type term, followed by incrementally adding other terms. Valid solutions are:

In [ ]:
pauli = +I       # Valid
pauli += 1.0     # Does not affect I 
pauli = 1.0 * I  # Valid
pauli += 1.0     # Does not affect I 
pauli = I.copy() # Valid
pauli += 1.0     # Does not affect I 
print(I)

The following will modify `I`, and should not be used:

In [ ]:
pauli = I     # Shallow
pauli += 1.0  # Modifies I
print(I)